In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
files = list()
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        files.append(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Netflix Dataset Analysis

In [ ]:
files[0]

In [ ]:
# import sys

# def read_file(file):
#     data = []
#     with open(file, "r") as f:
#         for line in f:
#             data.append(line.split(','))
#     return data

# print(read_file(files[0])[1:])
# npdata = np.array(read_file(files[0]))

In [ ]:
# print(npdata[0:10])

In [ ]:
# tvcount = 0 # select 'TV Show', count(type) from netflix where type = 'TV Show'
# mvcount = 0 # select 'Movie', count(type) from netflix where type = 'Movie'

# print(len(npdata))
# for x in npdata:
# #     print(x[1])
#     if 'TV Show' in x:
#         tvcount +=1
#     elif 'Movie' in x:
#         mvcount +=1

# print("TV Show : ",tvcount)
# print("Movies  : ",mvcount)

In [ ]:
pddata = pd.read_csv(files[0], parse_dates = ['date_added'])
(pddata.head(5))

# # numerical cols
# (pddata.describe())

# (pddata.info())

(pddata.tail(5))

In [ ]:
print(pddata['type'].head(10))

print(pddata['type'].describe())

In [ ]:
print(pddata[pddata['type'] == 'Movie']['type'].count)  # select 'Movie', count(type) from netflix where type = 'Movie'
print(pddata[pddata['type'] == 'TV Show']['type'].count)  # select 'TV Show', count(type) from netflix where type = 'TV Show'

In [ ]:
print(pddata['director'][pddata['director'].isna()].index)

print(pddata.columns)

In [ ]:
pddata.iloc[0]

In [ ]:
pddata.at[0,'type']

In [ ]:
# pd -> data structure [(1D)Series,(2D)DataFrame, (ND)Panel]
# (2D)DataFrame Tabular/Formated DataStructure

# for label, content in pddata.items():
#     print(label,content)

In [ ]:
pddata['rating'].describe()

In [ ]:
# select rating, count(*) as 'rating_count' from netflix group by rating
# TV MA 2863
# pddata.groupby('rating').describe()

rating_data = pddata.groupby('rating')

ratings = rating_data.agg(np.size)['show_id']
print(len(pddata))

tsize = len(pddata)

for label, content in ratings.items():
    ratingc = int(content)
    dataper = (ratingc / tsize) * 100
    print(label, format(dataper, '0.2f'))

In [ ]:
# get dummies categorical data into  the columns : values of data 0/1
rating_data = pd.get_dummies(pddata['rating'])
rating_data.head(10)

# ['G','NC', .....]
#     G   NC  ........
# 0   1   0  0 0 0  
# 1   0   1  0 0 0 



# for lable in rating_data.columns:
#     print(lable, rating_data[lable].sum())

In [ ]:
pddata['director'][pddata['director'].isna()].index

In [ ]:
directordata = pddata.drop(pddata['director'][pddata['director'].isna()].index)
print(directordata['director'].head())

In [ ]:
director_counts = directordata.groupby('director').agg(np.size)['show_id']
director_counts
# for lable, content in director_counts.items():
#     direc = int(content)
#     dataper = (direc / tsize) * 100
#     print(lable, format(dataper, '0.2f'))

In [ ]:
def split_genre(item):
    return item.split(',')

genre = pddata['listed_in'].apply(split_genre)

print(genre)

# not allow duplicate value
data_g = set()

for item in genre:
    for g in item:
        data_g.add(g.strip())

genere_list = list(data_g)
print(len(genere_list))
print(genere_list[0:5])

In [ ]:
# genre = pd.get_dummies(genere_list) 
# genre.head(10)

for genr in genere_list:
    pddata[genr] = np.zeros(len(pddata))

pddata[genere_list].tail(2)
for label, item in pddata['listed_in'].items():
    for genr in genere_list:
        if genr in item:
            pddata.at[label, genr] = 1.0
pddata[genere_list].tail(2)


pddata[(pddata['type'] == 'TV Show') & (pddata['Comedies'] == 1)]

In [ ]:
generes = {}
for genr in genere_list:
    generes[genr.strip()] = pddata[genr].sum()
print(generes)
pdgenre = pd.Series(data = generes , index = genere_list)


pdgenre

In [ ]:

# countrydata = pddata.drop(pddata['country'][pddata['country'].isnull()].index)

# countrydata = pddata.drop(pddata['country'][pddata['country'].isna()].index)

countrydata = pddata['country'].dropna()

def split_country(item):
    splidata = []
    try:
        splidata = item.split(',')
    except:
        splidata = np.nan
    return splidata

# inline function
# lambda x : x > 10

pdcountry = countrydata.apply(split_country)


# United States, India
unique_country = set()

for items in pdcountry:
    for item in items:
        unique_country.add(item.strip())

unique_country = list(unique_country)

unique_country.remove('')

country_list = {}
for items in countrydata:
    for item in unique_country:
        if item in items:
            if item in country_list.keys():
                country_list[item] += 1
            else:
                country_list[item] = 1

print(country_list)

In [ ]:
for country in country_list:
    pddata[country] = np.zeros(len(pddata))

pddata.drop(pddata['country'][pddata['country'].isna()].index, inplace = True)

print(len(pddata))
# pddata[genere_list].tail(2)
for label, item in pddata['country'].items():
    for country in country_list:
        if country in item:
            pddata.at[label, country] = 1.0



In [ ]:
# pddata.drop(unique_country, axis=1)

pddata['United States'][pddata['type'] == 'TV Show'].sum()


In [ ]:
pddata[unique_country][pddata['type'] == 'Movie'].sum()

In [ ]:
# Grouping Type with Duration

duration = pddata['duration'].unique()

In [ ]:
grouping_type_dur = pddata.groupby(['type','duration']).agg(np.size)
grouping_type_dur.rename(columns={"show_id": "group_by_type_dur"}, inplace= True)

grouping_type_dur.drop(grouping_type_dur.columns[1:],axis=1,inplace = True)

In [ ]:
grouping_type_dur.head(10)

In [ ]:
grouping_type_dur[grouping_type_dur['group_by_type_dur'] > 50 ].loc['TV Show']

In [ ]:
grouping_type_dur[grouping_type_dur['group_by_type_dur'] > 50 ].loc['Movie']

In [ ]:
grouping_type_dur[grouping_type_dur['group_by_type_dur'] > 50 ].loc['TV Show'].sort_values('group_by_type_dur',ascending = False)

In [ ]:
grouping_type_dur[grouping_type_dur['group_by_type_dur'] > 50 ].loc['Movie'].sort_values('group_by_type_dur',ascending = True)

In [ ]:
grouping_type_dur['season'] = np.zeros(len(grouping_type_dur))
grouping_type_dur['mint'] = np.zeros(len(grouping_type_dur))

grouping_type_dur

In [ ]:
grouping_type_dur.index

grouping_type_dur.reset_index(inplace=True)
grouping_type_dur.head()

In [ ]:
print(grouping_type_dur.head())

for index,item in grouping_type_dur['duration'].items():
    if 'Seasons' in grouping_type_dur.at[index,'duration']:
        grouping_type_dur.at[index, 'season'] = int(grouping_type_dur.at[index,'duration'].replace('Seasons','').strip())
    elif 'Season' in grouping_type_dur.at[index,'duration']:
        grouping_type_dur.at[index, 'season'] = int(grouping_type_dur.at[index,'duration'].replace('Season','').strip())
    elif 'min' in grouping_type_dur.at[index,'duration']:
        grouping_type_dur.at[index, 'mint'] = int(grouping_type_dur.at[index,'duration'].replace('min','').strip())

In [ ]:
grouping_type_dur

print(int(grouping_type_dur.at[index,'duration'].replace('Seasons','').strip()))


In [ ]:
grouping_type_dur[(grouping_type_dur['season'] >= 1) & (grouping_type_dur['season'] <= 3)].sum()

# print(grouping_type_dur['group_by_type_dur'].sum())

In [ ]:
grouping_type_dur[(grouping_type_dur['season'] == 1)]

In [ ]:
season13 = grouping_type_dur[(grouping_type_dur['season'] >= 1) & (grouping_type_dur['season'] <= 3)].sum()['group_by_type_dur']
total = grouping_type_dur[grouping_type_dur['type'] == 'TV Show']['group_by_type_dur'].sum()

print(format(season13/total, '.2%'))

In [ ]:
season45 = grouping_type_dur[(grouping_type_dur['season'] > 3)].sum()['group_by_type_dur']
total = grouping_type_dur[grouping_type_dur['type'] == 'TV Show']['group_by_type_dur'].sum()

print(format(season45/total, '.2%'))

In [ ]:
titlegrp = pddata.groupby(['title']).agg(np.size)
# print(titlegrp)

# titlegrp[titlegrp['type'] == 'TV Show']

In [ ]:
titlegrp.drop(titlegrp.columns[1:],axis=1,inplace=True)

In [ ]:
titlegrp.sort_values('show_id',ascending = False)

# Visualisation with Matplotlib

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
plt.figure(figsize= (16, 16))

# category based graphs
xdata = (pdgenre.index)[:10]
ydata = list(map(int,pdgenre.values[:10]))
plt.plot(xdata,ydata)
plt.show()

In [ ]:
# top 10 listed genre
top10genre = pdgenre.sort_values(ascending=True).head(10)

x = top10genre.index
y = top10genre.values

plt.figure(figsize= (16, 16))
plt.style.use('classic')
plt.plot(x,y)
plt.xlabel("Category")
plt.ylabel("Count")
plt.title("Category Vs Count")
plt.show()

In [ ]:
# top 10 listed genre
top10genre = pdgenre.sort_values(ascending=True).head(10)

x = top10genre.index
y = top10genre.values

yav = np.array(y).std()

plt.figure(figsize= (16, 16))
plt.style.use('default')
fig, ax = plt.subplots()
ax.bar(x, y, 0.35, yerr=yav, label='Category')
plt.xticks(x,rotation ='vertical')

fig.set_figheight(20)
fig.set_figwidth(20)
plt.xlabel("Category")
plt.ylabel("Count")
plt.title("Category Vs Count")
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(9, 3), sharey=True)
axs[0].bar(x, y)
axs[0].set_xticklabels(x,rotation ='vertical')
axs[1].scatter(x, y)
axs[1].set_xticklabels(x,rotation ='vertical')
axs[2].plot(x, y)
axs[2].set_xticklabels(x,rotation ='vertical')

fig.suptitle('Categorical Plotting')


In [ ]:
# country based graphs

countrydata = pd.Series(country_list)
x = countrydata.sort_values(ascending=False).head(10).index
y = countrydata.sort_values(ascending=False).head(10).values



fig, axs = plt.subplots(1, 1, figsize=(16, 16), sharey=True)
plt.style.use('classic')
plt.xlabel("Category")
plt.ylabel("Count")
plt.title("Category Vs Count")
axs.pie(y, labels = x,autopct='%1.2f%%')
plt.show()


In [ ]:
ustv = pddata['United States'][pddata['type'] == 'TV Show'].sum()
usmv = pddata['United States'][pddata['type'] == 'Movie'].sum()

print(ustv,usmv)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 16))
fig.subplots_adjust(wspace=0)
plt.style.use('default')
plt.xlabel("Category")
plt.ylabel("Count")
plt.title("Category Vs Count")
ax1.pie(y, labels = x,autopct='%1.2f%%')

usrange = [ustv,usmv]
colors = [[.1, .3, .5], [.1, .3, .3]]
xpos = 0
bottom = 0
width = 0.35

for j in range(len(usrange)):
    height = usrange[j]
    ax2.bar(xpos, height, width, bottom=bottom, color=colors[j])
    ypos = bottom + ax2.patches[j].get_height() / 2
    bottom += height
    ax2.text(xpos, ypos, "%d%%" % ((ax2.patches[j].get_height() * 100) /(ustv+usmv)),
             ha='center')
    
ax2.set_title('Type Based Data')
ax2.legend(('TV-Show', 'Movie'))
ax2.axis('off')
ax2.set_xlim(- 2.5 * width, 2.5 * width)

In [ ]:
from wordcloud import WordCloud

plt.subplots(figsize =(8,8))
wordcloud = WordCloud(
                            background_color = 'white',
                            width = 512,
                            height = 384
                        ).generate(" ".join(pddata['listed_in']))

plt.imshow(wordcloud)
plt.axis("off")
plt.savefig('graph.png')

plt.show()

In [ ]:
movie = pddata[pddata['type'] == 'Movie']
tv = pddata[pddata['type'] == 'TV Show']

data = pddata[['type', 'release_year']]
data = data.value_counts().to_frame()
data.reset_index(level=[0,1], inplace=True)
data = data.rename(columns = {0:'count'})
data = pd.concat([data[data['type'] == 'Movie'][:10], data[data['type']== 'TV Show'][:10]])

xmv = data[data['type'] == 'Movie'].sort_values(by=['release_year'],ascending=True)
xtv = data[data['type'] == 'TV Show'].sort_values(by=['release_year'],ascending=True)


fig, ax = plt.subplots()
ax.plot(xmv['release_year'], xmv['count'], label="Movie")
ax.plot(xtv['release_year'], xtv['count'], label="TV Show")
ax.legend()

plt.show()


In [ ]:
catwithyear = pddata.groupby(['release_year']).agg(np.sum)[pdgenre.index]
catwithyear.sort_values(by=['release_year'],ascending=False)

top10cat = catwithyear[catwithyear.columns[:10]].sort_values(by=['release_year'],ascending=False)

year = top10cat.head(10).index

fig, ax = plt.subplots()

for item in top10cat.head(10).columns[:10]:
    #print(top10cat[item].values)
    ax.plot(year, top10cat[item].values[:10], label=item)

ax.legend()

plt.show()


In [ ]:
countryyear = pddata.groupby(['release_year']).agg(np.sum)[country_list.keys()]
countryyear.sort_values(by=['release_year'],ascending=False)

top10country = countryyear[countryyear.columns[:10]].sort_values(by=['release_year'],ascending=False)

year = top10country.head(10).index

fig, ax = plt.subplots()

for item in top10country.head(10).columns[:10]:
    #print(top10cat[item].values)
    ax.plot(year, top10country[item].values[:10], label=item)

ax.legend()

plt.show()


In [ ]:
# top 10 Ratings
ratings = pddata['rating'].unique()
ratings


ratinglist = []
datalist = []
for rating in ratings:
    rat = (pddata[pddata['rating'] == rating]['rating'].value_counts().values)
    if bool(rat):
        ratinglist.append(rating)
        datalist.append(rat[0])

plt.figure(figsize= (16, 16))
plt.style.use('default')
plt.bar(ratinglist,datalist,width=0.35)
plt.xlabel("Ratings")
plt.ylabel("Count")
plt.title("Rating Vs Count")
plt.show()

In [ ]:
# top 10 directors
xdir = director_counts.sort_values(ascending=False)[:10].index
ydir = director_counts.sort_values(ascending=False)[:10].values


plt.figure(figsize= (16, 16))
plt.style.use('default')
plt.bar(xdir,ydir,width=0.35)
plt.xlabel("Director")
plt.ylabel("Count")
plt.title("Director Vs Count")
plt.show()

In [ ]:
import seaborn as sns



sns.heatmap(pddata[pdgenre.index[:10]].corr(),annot=True)
plt.show()

In [ ]:
ctlist = pd.Series(country_list)
plt.figure(figsize= (16, 16))

sns.heatmap(pddata[ctlist.index[:15]].corr(),annot=True)
plt.show()

In [ ]:
# Recommendation system 
# user    moneyheist: Thriller
# sumit    w : 1 visit count
#          r : 4 rating count

# market basket analysis -> Frequent Itemset Algorithm
# sumit milk,butter,bread
# sumit2 milk,bread,sugar
# sumit3 bread,butter,jam


ctlist = pd.Series(country_list)
plt.figure(figsize= (16, 16))

sns.heatmap(pddata[pddata['type'] == 'TV Show'][ctlist.index[:15]].corr(),annot=True)
plt.show()

In [ ]:
ctlist = pd.Series(country_list)
plt.figure(figsize= (16, 16))

sns.heatmap(pddata[pddata['type'] == 'Movie'][ctlist.index[:15]].corr(),annot=True)
plt.show()

In [ ]:
ctlist = pd.Series(country_list)
plt.figure(figsize= (16, 16))

sns.heatmap(pddata[pddata['type'] == 'Movie'][pdgenre.index[:15]].corr(),annot=True)
plt.show()

In [ ]:
ctlist = pd.Series(country_list)
plt.figure(figsize= (16, 16))

sns.heatmap(pddata[pddata['type'] == 'TV Show'][pdgenre.index[:15]].corr(),annot=True)
plt.show()